### Comments

- remove EXIF

# Upload Notebook
Uploads all files to Nakala

## 0. Initialization

In [ ]:
## install required packages

# create requirements.txt
! echo "pandas==1.1.4" > requirements_upload.txt
! echo "requests==2.27.1" >> requirements_upload.txt

# install requirements
! pip install -r requirements_upload.txt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os, ast
import csv, requests, json, time
import pandas as pd
from datetime import datetime

sys.path.insert(0, os.path.abspath('./nakalapyconnect/'))
import nklPushCorpus as npc

In [ ]:
data_pd = pd.read_csv('photos.csv')

n_photos = len(data_pd)
n_already_uploaded = sum(data_pd.Uploaded)
n_to_upload = n_photos - n_already_uploaded

print("{} photos have been identified: ".format(n_photos))
print("- {} photos have already been uploaded".format(n_already_uploaded))
print("- {} photos need to be uploaded".format(n_to_upload))

# Endpoint & Clef d'API
apiUrl = npc.myConst.API_URL
apiKey = npc.API_KEY_NKL

print('\nKEYS:')
print('URL:',apiUrl)
print('KEY:',apiKey,'\n')

if apiUrl == 'https://apitest.nakala.fr':
    is_api_test = True
    print('You are working on the TEST API.')
else:
    is_api_test = False
    print("/!\ YOU ARE WORKING ON THE MAIN API, ANY CHANGE IS PERMANENT!")
    
if is_api_test:
    link_base = "https://test.nakala.fr/"
else:
    link_base = "https://nakala.fr/"

## 1. Upload collection

### 1.1. Check if already uploaded collections

In [ ]:
# Create collection file
list_collections = list(set(','.join(list(data_pd['Collections'])).split(',')))
list_collections

collections = pd.DataFrame(index=list_collections, columns = ['uploaded', 'link', 'error', 'date']).reset_index()
collections = collections.rename({'index': 'collection'},axis=1)
collections['uploaded']=0
collections = collections.sort_values(by='collection').reset_index(drop=True)

collections = collections#.iloc[0:1]
print("{} collections have been found in the photos file".format(len(collections)))

In [ ]:
list(set(','.join(list(data_pd['Collections'])).split(',')))

In [ ]:
# find an existing version of collections.csv

name_old_collection_file = 'collections.csv'

try:
    collections_old = pd.read_csv(name_old_collection_file)
    print("an existing collecion file has been found, containing {} collections".format(len(collections_old)))
except:
    collections_old = pd.DataFrame()
    print("/!\ no existing collecion file named '{}' has been found!".format(name_old_collection_file))
    print("If you already uploaded collections, make sure the file hasn't been moved or renamed")

In [ ]:
# isolate new collections

if len(collections_old) > 0:
    merged = pd.merge(collections_old["collection"], collections, on='collection', how='outer', indicator=True)
    collections_new = merged[merged._merge == 'left_only']
    collections_new = collections_new.drop("_merge", axis=1)
    
else:
    collections_new = collections

print("{} new collections have been found".format(len(collections_new)))

In [ ]:
# append new collections 
collections_updated = collections_old.append(collections_new)

print("The new file contains {} new collections, including {} existing collections and {} new collections"\
      .format(collections_updated.shape[0],
              collections_old.shape[0],
              collections_new.shape[0]))

print("{} collections have already been uploaded".format(sum(collections_updated['uploaded'] == 1)))

In [ ]:
collections = collections_updated.set_index('collection')
collections

In [ ]:
collections.to_csv('collections.csv')

### 1.2. Upload collections not previously uploaded

In [ ]:
dict_collection_handles = {}

for collection in collections.index:
    
    if collections.loc[collection, 'uploaded'] == 0:
    
        # création d'un dictionnaire dicoBody contenant le status, un nouveau title et la lang désirée
        dicoBody = {}
        dicoBody["status"] = "public"
        dicoBody["metas"] = []
        metaTitle={"value": collection,
          "lang": "fr",
          "typeUri": "http://www.w3.org/2001/XMLSchema#string",
          "propertyUri": "http://nakala.fr/terms#title"
          }
        dicoBody["metas"].append(metaTitle)

        # appel de la fonction post_collection
        response = npc.post_collections(dicoBody)

        if response.status_code == 201:
            response = json.loads(response.text)
            collections.loc[collection, 'uploaded'] = 1
            collections.loc[collection, 'link'] = link_base+'collection/'+response['payload']['id']
            collections.loc[collection, 'date'] = str(datetime.now())[0:19]
            print("Collection '{}' successfully uploaded: {}".format(collection,
                                                                     collections.loc[collection, 'link']))

        else:
            collections.loc[collection, 'error'] = str(response.text)
            print("Collection {} not uploaded! EROR: {}".format(collection,
                                                                response))

        collections.to_csv('collections.csv')

In [ ]:
n_not_uploaded_collections = sum(collections.uploaded == 0)

if n_not_uploaded_collections == 0:
    print("All collections have been uploaded! Proceed to next step.")
else:
    print("/!\ {} collections have not been uploade! Rerun previous cell.".format(n_not_uploaded_collections))

## 2. Upload photos

In [ ]:
# Gather collection handles
for collection in collections.index:
    if collections.loc[collection, 'uploaded'] ==1:
        dict_collection_handles[collection] = collections.loc[collection, 'link'].replace(link_base+'collection/', '')

In [ ]:
## UPLOAD DES DONNEES UNE PAR UNE:


status = 'published'

nakalaTypeDict = {
    "Image": "http://purl.org/coar/resource_type/c_c513",
}

nakalaCollectionDict = dict_collection_handles

# 1.1 Lecture du fichier CSV
with open('photos.csv', newline='') as f:
    reader = csv.reader(f)
    dataset = list(reader)
dataset.pop(0) # suppression des titres des colonnes

# 2. Parcours des différentes lignes du fichier
for num, data in enumerate(dataset):
    
    t0 = time.time()
    title = data[1]
    filenames = data[2].split(';')
    description = data[3]
    authors = list(filter(None,data[4].split(';')))
    date = data[5]
    keywords = list(filter(None,data[6].split(',')))
    datatype = data[7]
    license = data[8]
    nakalaCollections = list(filter(None,data[9].split(',')))
    is_uploaded = data[11]
        
    if is_uploaded == '0': # on n'upload que si pas déjà uploadé
        
        if set(nakalaCollections).issubset(dict_collection_handles.keys()):

            # 2.1. Récupération des infos disponibles sur la donnée à créer

            print('CREATION DE LA DONNEE ' + str(num) + " : " + title)

            # 2.2. Envoi des fichiers à l'API
            files = [] # variable pour stocker les informations retournées en JSON par l'API à chaque upload.
            for filename in filenames: # on parcours l'ensemble des fichiers d'une donnée


                #### INSERT: replace new filename with EXIF / Compression (temp compressed)
                # ============================


                print('Envoi du fichier ' + filename + '...') # on affiche un message pour le suivi de l'upload
                goToNextData = False
                # écriture de la requête à l'API (ne contient pas de body en JSON, mais un fichier et un clef d'API)
                payload={}
                postfiles=[('file',(filename,open(filename, 'rb')))]
                headers = {'X-API-KEY': apiKey }
                # appel à l'API pour uploader le fichier
                response = requests.request("POST", apiUrl + '/datas/uploads', headers=headers, data=payload, files=postfiles)
                # si l'upload s'est bien passé, on stocke les informations retournés par l'API dans la variable 'files'
                if ( 201 == response.status_code ):
                    # avant de stocker les informations retournées par l'API sur le fichier, on y ajoute une date d'embargo
                    file = json.loads(response.text)
                    file["embargoed"] = time.strftime("%Y-%m-%d") # on renseigne la date du jour si pas de date d'embargo renseignée
                    files.append(file)
                else:
                    # une erreur s'est produite avec un upload
                    print ("Certains fichiers n'ont pas pu être envoyés, on passe à la donnée suivante...") # on affiche un message d'erreur
                    goToNextData = True # on stocke dans cette variable qu'il y a eu un problème
                    break # on arrête l'upload des autres fichiers de la donnée
            if goToNextData: continue # on passe à la donnée suivante si

            # 2.4. Reconstruction des métadonnées
            metas = [] # on stocke dans cette variable l'ensemble des métadonnées dans le format attendu

            # la métadonnée type (obligatoire)
            metaType = {
                "value": nakalaTypeDict[datatype], # on insère ici le contenu de la colonne "datatype" mappé sur l'URI correspondante (un seul type par donnnée)
                "typeUri": "http://www.w3.org/2001/XMLSchema#anyURI", # on indique ici que la valeur renseignée est de type URI
                "propertyUri": "http://nakala.fr/terms#type" # on indique ici le champ "type" issu du vocabulaire NAKALA
            }
            metas.append(metaType) # ajout de la métadonnée dans le tableau "metas"

            # la métadonnée titre (obligatoire)
            metaTitle = {
                "value": title, # on insère ici le contenu de la colonne "nakala:title (fr)" (un seul titre fr par donnnée)
                "lang": "fr", # on indique ici la langue du titre (cf. ISO-639-1 ou ISO-639-3 pour les langues moins courantes)
                "typeUri": "http://www.w3.org/2001/XMLSchema#string", # on indique ici que la valeur renseignée est une chaîne de caractères
                "propertyUri": "http://nakala.fr/terms#title" # on indique ici le champ "title" issu du vocabulaire de NAKALA
            }
            metas.append(metaTitle)

            # les métadonnées auteurs (obligatoire pour une donnée publiée)
            if not authors: # si vide, on ajoute une métadonnée nakala:creator "anonyme"
               metaAuthor = {
                  "value": None, # sic. None sans guillemet veut dire "null" en Python
                  "propertyUri": "http://nakala.fr/terms#creator" # on indique ici le champ "creator" issu du vocabulaire de NAKALA
               }
               metas.append(metaAuthor)
            else:    
                for author in authors: # la colonne "nakala:creator" peut comporter plusieurs valeurs qu'on parcours une à une
                       # pour chaque valeur, on sépare le nom et le prénom pour construire la métadonnée nakala:creator
                    surnameGivennameORCID = author.split(',')
                    metaAuthor = {
                        "value": { # la valeur de cette métadonnée n'est pas une simple chaîne de caractères, mais un objet composé d'au minimum deux propriétées (givenname et surname)
                            "givenname": surnameGivennameORCID[1], # on insère ici le prénom
                            "surname": surnameGivennameORCID[0] # on insère ici le nom de famille
                        },
                        "propertyUri": "http://nakala.fr/terms#creator" # on indique ici le champ "creator" issu du vocabulaire de NAKALA
                    }
                    # on ajoute le numéro ORCID (si présent)
                    if len(surnameGivennameORCID) == 3:
                        metaAuthor["value"]["orcid"] = surnameGivennameORCID[2] # on insère ici le numéro ORCID
                    # ajout de la métadonnée
                    metas.append(metaAuthor)

            # la métadonnée date de création (obligatoire pour une donnée publiée)
            if not date: # si vide, on ajoute une métadonnée "nakala:created" "inconnue"
                metaCreated = {
                    "value": None,
                    "propertyUri": "http://nakala.fr/terms#created"
                }
            else:
                metaCreated = {
                    "value": date, # on insère ici le contenu de la colonne "nakala:created" (une seule date de création par donnée)
                    "typeUri": "http://www.w3.org/2001/XMLSchema#string",
                    "propertyUri": "http://nakala.fr/terms#created"
                }
            metas.append(metaCreated)

            # la métadonnée licence (obligatoire pour une donnée publiée)
            metaLicense = {
                "value": license, # On insère ici le contenu de la colonne "nakala:license" (une seule licence par donnée)
                "typeUri": "http://www.w3.org/2001/XMLSchema#string",
                "propertyUri": "http://nakala.fr/terms#license"
            }
            metas.append(metaLicense)

            # la métadonnée description (facultative)
            metaDescription = {
                "value": description, # on insère ici le contenu de la colonne "dcterms:description (fr)" (une seule description par donnée)
                "lang": "fr",
                "typeUri": "http://www.w3.org/2001/XMLSchema#string",
                "propertyUri": "http://purl.org/dc/terms/description" # notez qu'il ne s'agit plus ici d'une propriété issue du vocabulaire NAKALA, mais du vocabulaire Dcterms
            }
            metas.append(metaDescription)

            # les métadonnées mots-clés (facultatives)
            for keyword in keywords: # on parcours les valeurs de la colonne "dcterms:subject (fr)"
                metaKeyword = {
                    "value": keyword, # on insère ici la valeur d'un mot-clé
                    "lang": "fr",
                    "typeUri": "http://www.w3.org/2001/XMLSchema#string",
                    "propertyUri": "http://purl.org/dc/terms/subject" # notez qu'il ne s'agit plus ici d'une propriété issue du vocabulaire NAKALA, mais du vocabulaire Dcterms
                }
                metas.append(metaKeyword)


            # 2.6. Reconstruction des collections
            collectionsIds = []
            for nakalaCollection in nakalaCollections:
                collectionsIds.append(nakalaCollectionDict[nakalaCollection])

            # 3. Envoi de la donnée à NAKALA
            postdata = { # variable contenant le contenu de la requête
                "status" : status, # on publie directement les données
                "files" : files,
                "metas" : metas,
                #"rights": rights,
                "collectionsIds": collectionsIds
            }
            content = json.dumps(postdata) # serialisation du contenu en JSON
            headers = { # header de la requête
              'Content-Type': 'application/json',
              'X-API-KEY': apiKey,
            }
            response = requests.request("POST", apiUrl + '/datas', headers=headers, data=content) # requête à l'API
            if ( 201 == response.status_code ): # on obtient un code 201 si tout s'est bien passé
                parsed = json.loads(response.text) # on parse la réponse de l'API
                print('La donnée ' + str(num) + ' a bien été créée : ' + link_base + parsed["payload"]["id"]) # affichage d'un message de succès            

                data_pd.iloc[num,11] = 1
                data_pd.iloc[num,12] = str(datetime.now())[0:19]
                data_pd.iloc[num,13] = link_base + parsed["payload"]["id"]
                data_pd.iloc[num,0]  = status
                data_pd.iloc[num,14] = ''
                data_pd.to_csv('photos.csv', index=False)
                
                t1 = time.time()
                print("photos.csv successfully updated! Upload took: {:.2f} seconds".format(t1-t0))
                print('\n')

            else:
                print("Une erreur {} s'est produite !".format(response.status_code))
                json_resp = json.loads(response.text)
                print(json_resp)

                data_pd.iloc[num,12] = str(datetime.now())[0:19]
                data_pd.iloc[num,14] = str(response.text)
                data_pd.to_csv('photos.csv', index=False)
                t1 = time.time()
                print("photos.csv uploaded with error. Failed upload took: {:.2f} seconds".format(t1-t0))
                print('\n')

        
        else:
            print("{} cannot be uploaded: missing one of {} keys".format(title, nakalaCollections))

    else:
        print("{} already uploaded".format(title))